In [1]:
#import class used for simulations
import numpy as np
from dicty_population_sims.dictyPop import dicty_pop_opto_active_1D
import matplotlib.pyplot as plt

In [2]:
#the method ive been using to extract frequencies/wavenumbers
def get_out_k(ipt):
    ffts = np.fft.fft(ipt-np.average(ipt,axis=0),axis=0)
    idx = np.argmax(np.abs(np.average(ffts,1)))
    return 2*np.pi*idx/ffts.shape[0]

In [3]:
#parameters for the grid and the individual cell dynamics
#note: need agent_dim*box_size_x <= num_agents right now (no stacking yet)
grid_params_1D = {
        'dx' : 0.1,
        'D_sig' : 1,
        'box_size_x' : 250,
        'box_size_y' : .5,
        'agent_dim' : .5,
        'num_agents' : 500,
        }

cell_params={
        'c0' : 1,
        'a' : .058,
        'gamma' : 0.5,
        'Kd' : 10**(-5),
        'sigma' : .15,
        'epsilon' : 0.2,
        'cStim' : 100,
        'J' : 10,
        'rho' :  1,
        'D' : 1000,
        'a0' : 1,
        'af' : 0
       }

In [4]:
#set the 0th cell in the array to have its dynamics determined by the external "optogenetic" signal 
mask = np.ones(grid_params_1D['num_agents'],dtype = int)
mask[0] = 0

In [5]:
#set the frequency of the driving optogenetic signal
w = 2*np.pi/30

In [ ]:
#initialize the simulated population
pop = dicty_pop_opto_active_1D(2000,1,g_params = grid_params_1D,c_params = cell_params,progress_report = False)
#set the "optogenetic" mask for the population
pop.setMask(mask)
#define the input signal
t = np.linspace(0,pop.T,pop.Tsteps)
A_in = 2*np.sin(w*t)
R_in = np.zeros(pop.Tsteps)
#run the population
#Warning: this takes a while
pop.run(A_in,R_in)

In [ ]:
#extract the activator trace
a_out = pop.A_saved
#take remove the cells close to the input and the initial half of the timetrace
a_out_clean = a_out[-250:,-1000:]

In [ ]:
#get the average frequency
k = get_out_k(a_out_clean)
print('calculated wavelength :',2*np.pi/k)

In [ ]:
#plot of activity of all cells at a given time:
plt.plot(a_out[:,1000])
#note: at this input frequency we get this weird double-beating pattern
#im still not sure what the deal with this is yet, i need to do more testing

In [ ]:
#plot of activity of a single cell as a function of time (limited time window):
plt.plot(a_out[250,250:750])

In [ ]:
#kymograph of the activator dynamics
fig, ax = plt.subplots(figsize=(15, 50))
ax.imshow(a_out, interpolation='nearest')

In [ ]:
#power spectrum
a_out_clean_k = np.fft.fft(a_out_clean-np.average(a_out_clean,axis=0),axis=0)
plt.plot(np.abs(np.average(a_out_clean_k,1)))